In [7]:
import os, json, glob
from pathlib import Path
from typing import List, Dict, Any, Iterable
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# ===== 설정 =====
INPUT_DIR = "./COMPANY_DATA/cto"                 # 텍스트 문서 폴더(.txt/.md/.html)
OUTPUT_JSONL = "./qa_dataset.jsonl"  # 결과 저장 경로
PAIRS_PER_CHUNK = 4                  # 조각당 생성할 Q/A 개수
CHUNK_SIZE = 2500                    # 문자 기준 조각 크기
CHUNK_OVERLAP = 250                  # 조각 겹침

def query_setting_for_internal_documents():
    return ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={"response_format": {"type": "json_object"}},
    )

llm = query_setting_for_internal_documents()

# ===== 간단 유틸 =====
def iter_text_files(root: str) -> Iterable[str]:
    for pattern in ("**/*.txt", "**/*.md", "**/*.html", "**/*.htm"):
        yield from (p for p in glob.glob(os.path.join(root, pattern), recursive=True) if os.path.isfile(p))

def read_text(path: str) -> str:
    # 인코딩 오류만 가볍게 우회
    try:
        return Path(path).read_text(encoding="utf-8")
    except UnicodeDecodeError:
        return Path(path).read_text(encoding="cp949", errors="ignore")

def chunk_text(s: str, size: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> List[str]:
    if not s:
        return []
    chunks = []
    i, n = 0, len(s)
    while i < n:
        start = max(0, i - (overlap if i else 0))
        end = min(n, i + size)
        chunks.append(s[start:end])
        i += size if i == 0 else (size - overlap)
    return chunks

# ===== 프롬프트 (사내 실무형) =====
SYSTEM = SystemMessage(content=(
    "너는 기업 내부 문서 전용 Q&A 데이터셋을 생성하는 어시스턴트다. "
    "주어진 '문서 조각'만을 근거로, 사내 직원이 사내 문서 챗봇에게 할 법한 실제 실무형 질문과 그에 대한 정답을 생성하라. "
    "외부 지식을 섞지 말고, 문서 조각에 답이 명시적으로 존재하는 질문만 만들 것."
))

USER_TMPL = (
    "다음은 회사 내부 문서의 일부다. 이 조각만 보고 정확히 답할 수 있는 "
    "실무형 사내 질문/정답 페어를 최대 {k}개 생성하라.\n\n"
    "요구사항:\n"
    "- 질문은 사내 직원이 문서 챗봇에 묻는 자연스러운 톤(예: 설정/권한/절차/에러핸들링/운영규칙/SOP 등)\n"
    "- 답변은 반드시 문서 조각에 근거하여 구체적이고 정확하게 작성\n"
    "- 외부 지식 금지, 조각에 근거 없는 질문 금지\n"
    "- 출력은 JSON 객체 하나로만 반환\n\n"
    "문서 조각:\n\n{chunk}\n\n"
    "출력 스키마 예시:\n"
    "{{\"pairs\": [{{\"question\": \"...\", \"answer\": \"...\"}}]}}"
)

def generate_pairs(chunk: str, k: int) -> List[Dict[str, str]]:
    messages = [SYSTEM, HumanMessage(content=USER_TMPL.format(chunk=chunk, k=k))]
    # 최소한의 재시도(네트워크 흔들림 대비). 과도한 검증 제거.
    for _ in range(2):
        try:
            resp = llm.invoke(messages)
            data = json.loads(resp.content)
            return data.get("pairs", [])
        except Exception:
            pass
    # 실패 시 빈 리스트
    return []

def save_jsonl(rows: List[Dict[str, Any]], path: str):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, "a", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def build_dataset(input_dir: str, out_path: str):
    total = 0
    for fp in iter_text_files(input_dir):
        text = read_text(fp).strip()
        if not text:
            continue
        for chunk in chunk_text(text):
            pairs = generate_pairs(chunk, PAIRS_PER_CHUNK)
            if not pairs:
                continue
            rows = [{"question": p["question"], "reference": [chunk], "answer": p["answer"]}
                    for p in pairs if "question" in p and "answer" in p]
            if rows:
                save_jsonl(rows, out_path)
                total += len(rows)
    print(f"✅ 생성 완료: {total}개 저장 → {out_path}")

if __name__ == "__main__":
    build_dataset(INPUT_DIR, OUTPUT_JSONL)

✅ 생성 완료: 64개 저장 → ./qa_dataset.jsonl
